In [1]:
from haystack.telemetry import tutorial_running

tutorial_running(36)

In [2]:
from getpass import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")
if "SERPERDEV_API_KEY" not in os.environ:
    os.environ["SERPERDEV_API_KEY"] = getpass("Enter Serper Api key: ")

In [3]:
from haystack.dataclasses import Document

documents = [
  Document(
    content="""
Munich, the vibrant capital of Bavaria in southern Germany, exudes a perfect blend of rich cultural
heritage and modern urban sophistication. Nestled along the banks of the Isar River, Munich is renowned
for its splendid architecture, including the iconic Neues Rathaus (New Town Hall) at Marienplatz and
the grandeur of Nymphenburg Palace. The city is a haven for art enthusiasts, with world-class museums like the
Alte Pinakothek housing masterpieces by renowned artists. Munich is also famous for its lively beer gardens, where
locals and tourists gather to enjoy the city's famed beers and traditional Bavarian cuisine. The city's annual
Oktoberfest celebration, the world's largest beer festival, attracts millions of visitors from around the globe.
Beyond its cultural and culinary delights, Munich offers picturesque parks like the English Garden, providing a
serene escape within the heart of the bustling metropolis. Visitors are charmed by Munich's warm hospitality,
making it a must-visit destination for travelers seeking a taste of both old-world charm and contemporary allure.
"""
  )
]

from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator

prompt_template = [
  ChatMessage.from_user("""
Answer the following query given the documents.
If the answer is not contained within the documents reply with 'no_answer'
Query: {{query}}
Documents:
{% for document in documents %}
  {{document.content}}
{% endfor %}
""")
]

prompt_builder = ChatPromptBuilder(template=prompt_template)
llm = OpenAIChatGenerator(model='gpt-4o-mini')

In [4]:
from haystack.components.websearch.serper_dev import SerperDevWebSearch

prompt_for_websearch = [ChatMessage.from_user("""
Answer the following query given the documents retrieved from the web.
Your answer shoud indicate that your answer was generated from websearch.

Query: {{query}}
Documents:
{% for document in documents %}
  {{document.content}}
{% endfor %}
""")]

websearch = SerperDevWebSearch()
prompt_builder_for_websearch = ChatPromptBuilder(template=prompt_for_websearch)
llm_for_websearch = OpenAIChatGenerator(model='gpt-4o-mini')

In [5]:
from haystack.components.routers import ConditionalRouter

routes = [
    {
        "condition": "{{'no_answer' in replies[0].text}}",
        "output": "{{query}}",
        "output_name": "go_to_websearch",
        "output_type": str,
    },
    {
        "condition": "{{'no_answer' not in replies[0].text}}",
        "output": "{{replies[0].text}}",
        "output_name": "answer",
        "output_type": str,
    },
]

router = ConditionalRouter(routes)

In [6]:
from haystack import Pipeline

pipe = Pipeline()
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)
pipe.add_component("router", router)
pipe.add_component("websearch", websearch)
pipe.add_component("prompt_builder_for_websearch", prompt_builder_for_websearch)
pipe.add_component("llm_for_websearch", llm_for_websearch)

pipe.connect("prompt_builder.prompt", "llm.messages")
pipe.connect("llm.replies", "router.replies")
pipe.connect("router.go_to_websearch", "websearch.query")
pipe.connect("router.go_to_websearch", "prompt_builder_for_websearch.query")
pipe.connect("websearch.documents", "prompt_builder_for_websearch.documents")
pipe.connect("prompt_builder_for_websearch", "llm_for_websearch")

🚅 Components
  - prompt_builder: ChatPromptBuilder
  - llm: OpenAIChatGenerator
  - router: ConditionalRouter
  - websearch: SerperDevWebSearch
  - prompt_builder_for_websearch: ChatPromptBuilder
  - llm_for_websearch: OpenAIChatGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.messages (List[ChatMessage])
  - llm.replies -> router.replies (List[ChatMessage])
  - router.go_to_websearch -> websearch.query (str)
  - router.go_to_websearch -> prompt_builder_for_websearch.query (str)
  - websearch.documents -> prompt_builder_for_websearch.documents (List[Document])
  - prompt_builder_for_websearch.prompt -> llm_for_websearch.messages (List[ChatMessage])

In [7]:
pipe.draw("media/pipe.png")

In [8]:
query = "Where is Munich?"

result = pipe.run({"prompt_builder": {"query": query, "documents": documents}, "router": {"query": query}})

# Print the `answer` coming from the ConditionalRouter
print(result["router"]["answer"])

Munich is the vibrant capital of Bavaria in southern Germany, located along the banks of the Isar River.


In [9]:
query = "How many people live in Munich?"

result = pipe.run({"prompt_builder": {"query": query, "documents": documents}, "router": {"query": query}})

# Print the `replies` generated using the web searched Documents
print(result["llm_for_websearch"]["replies"])

[ChatMessage(content='As of May 31, 2024, Munich has a population of approximately 1,594,632 inhabitants. This makes it the most populous city in Bavaria and the third largest city in Germany.', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={'model': 'gpt-4o-mini-2024-07-18', 'index': 0, 'finish_reason': 'stop', 'usage': {'completion_tokens': 42, 'prompt_tokens': 378, 'total_tokens': 420, 'completion_tokens_details': CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), 'prompt_tokens_details': PromptTokensDetails(audio_tokens=0, cached_tokens=0)}})]


In [10]:
result['websearch']['links']

['https://en.wikipedia.org/wiki/Munich',
 'https://www.ricksteves.com/watch-read-listen/read/articles/munich-a-metropolis-with-smalltown-charm',
 'https://worldpopulationreview.com/cities/germany/munich',
 'https://www.britannica.com/place/Munich-Bavaria-Germany',
 'https://eurocities.eu/cities/munich/',
 'https://www.munich-business.eu/en/business-location/key-figures.html',
 'https://en.wikipedia.org/wiki/Demographics_of_Munich',
 'https://www.macrotrends.net/cities/204371/munich/population',
 'https://www.coe.int/en/web/interculturalcities/munich',
 'https://www.statista.com/statistics/505774/munich-population/']